In [1]:
import numpy as np
import json
import pickle
import math

In [25]:
path0 = '/home/daodao/OpenPCDet-hova88/data/nuscenes/v1.0-trainval/v1.0-trainval/'
j_sample_data = json.load(open(path0 + "sample_data.json", "rb"))
j_sample_annotation = json.load(open(path0 + "sample_annotation.json", "rb"))
j_sample = json.load(open(path0 + "sample.json", "rb"))
j_scene = json.load(open(path0 + "scene.json", "rb"))
j_sensor = json.load(open(path0 + "sensor.json", "rb"))
j_visibility = json.load(open(path0 + "visibility.json", "rb"))
j_map = json.load(open(path0 + "map.json", "rb"))
j_log = json.load(open(path0 + "log.json", "rb"))
j_instance = json.load(open(path0 + "instance.json", "rb"))
j_ego_pose = json.load(open(path0 + "ego_pose.json", "rb"))
j_category = json.load(open(path0 + "category.json", "rb"))
j_calibrated_sensor = json.load(open(path0 + "calibrated_sensor.json", "rb"))
j_attribute = json.load(open(path0 + "attribute.json", "rb"))

In [37]:
def pr(a):
    print(a[0].keys())
    print(len(a))
    print(a[100])
# pr(attribute)
# for i in sample_data:
#     if 'bin' in i['filename']:
#         print(i)
#         break
print(j_calibrated_sensor[0])
print(j_sensor)

{'token': '7781065816974801afc4dcdaf6acf92c', 'sensor_token': '47fcd48f71d75e0da5c8c1704a9bfe0a', 'translation': [3.412, 0.0, 0.5], 'rotation': [0.9999984769132877, 0.0, 0.0, 0.0017453283658983088], 'camera_intrinsic': []}
[{'token': '725903f5b62f56118f4094b46a4470d8', 'channel': 'CAM_FRONT', 'modality': 'camera'}, {'token': 'ce89d4f3050b5892b33b3d328c5e82a3', 'channel': 'CAM_BACK', 'modality': 'camera'}, {'token': 'a89643a5de885c6486df2232dc954da2', 'channel': 'CAM_BACK_LEFT', 'modality': 'camera'}, {'token': 'ec4b5d41840a509984f7ec36419d4c09', 'channel': 'CAM_FRONT_LEFT', 'modality': 'camera'}, {'token': '2f7ad058f1ac5557bf321c7543758f43', 'channel': 'CAM_FRONT_RIGHT', 'modality': 'camera'}, {'token': 'ca7dba2ec9f95951bbe67246f7f2c3f7', 'channel': 'CAM_BACK_RIGHT', 'modality': 'camera'}, {'token': 'dc8b396651c05aedbb9cdaae573bb567', 'channel': 'LIDAR_TOP', 'modality': 'lidar'}, {'token': '47fcd48f71d75e0da5c8c1704a9bfe0a', 'channel': 'RADAR_FRONT', 'modality': 'radar'}, {'token': '23

In [ ]:
for i in instance:
    if i['token'] == '5e2b6fd1fab74d04a79eefebbec357bb':
        print(i)
for i in category:
    if i['token'] == '85abebdccd4d46c7be428af5a6173947':
        print(i)

{'token': '5e2b6fd1fab74d04a79eefebbec357bb', 'category_token': '85abebdccd4d46c7be428af5a6173947', 'nbr_annotations': 13, 'first_annotation_token': '173a50411564442ab195e132472fde71', 'last_annotation_token': '2cd832644d09479389ed0785e5de85c9'}
{'token': '85abebdccd4d46c7be428af5a6173947', 'name': 'movable_object.trafficcone', 'description': 'All types of traffic cone.'}


In [2]:
def rpy2quaternion(roll, pitch, yaw):
    x=math.sin(pitch/2)*math.sin(yaw/2)*math.cos(roll/2)+math.cos(pitch/2)*math.cos(yaw/2)*math.sin(roll/2)
    y=math.sin(pitch/2)*math.cos(yaw/2)*math.cos(roll/2)+math.cos(pitch/2)*math.sin(yaw/2)*math.sin(roll/2)
    z=math.cos(pitch/2)*math.sin(yaw/2)*math.cos(roll/2)-math.sin(pitch/2)*math.cos(yaw/2)*math.sin(roll/2)
    w=math.cos(pitch/2)*math.cos(yaw/2)*math.cos(roll/2)-math.sin(pitch/2)*math.sin(yaw/2)*math.sin(roll/2)
    return [w, x, y, z]


In [62]:
def yaw_to_quart(yaw):
    R = np.array([math.cos(yaw), -math.sin(yaw), 0, math.sin(yaw), math.cos(yaw), 0, 0, 0, 1]).reshape(3, 3)

    w = math.sqrt(1 + R[0][0] + R[1][1] + R[2][2]) / 2
    x = (R[2][1] - R[1][2]) / (4 * w)
    y = (R[0][2] - R[2][0]) / (4 * w)
    z = (R[1][0] - R[0][1]) / (4 * w)

    return [w, x, y, z]

In [53]:
from pyquaternion import Quaternion

def quaternion_yaw(q):
    """
    Calculate the yaw angle from a quaternion.
    Note that this only works for a quaternion that represents a box in lidar or global coordinate frame.
    It does not work for a box in the camera frame.
    :param q: Quaternion of interest.
    :return: Yaw angle in radians.
    """

    # Project into xy plane.
    v = np.dot(q.rotation_matrix, np.array([1, 0, 0]))

    # Measure yaw using arctan.
    yaw = np.arctan2(v[1], v[0])

    return yaw

In [61]:
w, x, y, z = rpy2quaternion(0, 0, -2)
print(x,y,z,w)
quaternion_yaw(Quaternion(w,x,y,z))

0.0 0.0 -0.8414709848078965 0.5403023058681398


-1.9999999999999998

In [3]:
path1 = '/home/daodao/OpenPCDet-hova88/data/nucustom/v1.0-trainval/v1.0-trainval/'
#cdi_infos = pickle.load(open(path1 + '../cdi_infos_1sweeps_train_1024_1901.pkl', 'rb'))
cdi_infos = pickle.load(open(path1 + '../cdi_infos_1sweeps_train_1104_9657.pkl', 'rb'))


In [10]:
def info2json(info):
    lidar_path = info['lidar_path']
    token = lidar_path.replace('/', '_').replace('-', '').replace('.', '_')

    sample = {}
    # dict_keys(['token', 'timestamp', 'prev', 'next', 'scene_token'])
    sample['token'] = 'sample_token_' + token
    sample['timestamp'] = 0
    sample['prev'] = ''
    sample['next'] = ''
    sample['scene_token'] = 'scene_token_0'
    
    # dict_keys(['token', 'sample_token', 'ego_pose_token', 'calibrated_sensor_token', 'timestamp', 
    # 'fileformat', 'is_key_frame', 'height', 'width', 'filename', 'prev', 'next'])
    sample_data = [{'token':'sample_data_token_' + token,
                    'sample_token':'sample_token_' + token,
                    'ego_pose_token':'ego_pose_token',
                    'calibrated_sensor_token':'calibrated_sensor_lidar_top_token',
                    'timestamp':0,
                    'fileformat':'pcd',
                    'is_key_frame':True,
                    'height':0, 'width':0,
                    'filename':lidar_path,
                    'prev':0, 'next':0}, 
                   {'token':'sample_data_cam_token_' + token,
                    'sample_token':'sample_token_' + token,
                    'ego_pose_token':'ego_pose_token',
                    'calibrated_sensor_token':'calibrated_sensor_cam_front_token',
                    'timestamp':0,
                    'fileformat':'jpg',
                    'is_key_frame':True,
                    'height':0, 'width':0,
                    'filename':'',
                    'prev':0, 'next':0    
                    }]
    
    # dict_keys(['token', 'sample_token', 'instance_token', 'visibility_token', 'attribute_tokens', 
    # 'translation', 'size', 'rotation', 'prev', 'next', 'num_lidar_pts', 'num_radar_pts'])
    sample_annotation = [] #[{'token':'sample_annotation_token_' + token + '_null',
                        #   'sample_token':'sample_token_' + token ,
                        #   'instance_token':'instance_token_' + token + '_null',
                        #   'visibility_token':'4',
                        #   'attribute_tokens':[],
                        #   'translation':[0.0, 0.0, 0.0],
                        #   'size':[0.0, 0.0, 0.0],
                        #   'rotation':[0.0, 0.0, 0.0, 1],
                        #   'prev':'',
                        #   'next':'',
                        #   'num_lidar_pts':0,
                        #   'num_radar_pts':0}]

    # dict_keys(['token', 'category_token', 'nbr_annotations', 'first_annotation_token', 'last_annotation_token'])
    instance = [] #[{'token':'instance_token_' + token + '_null',
        #   'category_token':'movable_object.trafficcone',
        #   'nbr_annotations':0,
        #   'first_annotation_token':'',
        #   'last_annotation_token':''}]

    for i, gt in enumerate(info['gt_boxes']):
        sa = {}
        sa['token'] = 'sample_annotation_token_' + token + '_' + str(i) 
        sa['sample_token'] = 'sample_token_' + token 
        sa['instance_token'] = 'instance_token_' + token + '_' + str(i) 
        sa['visibility_token'] = '4'
        sa['attribute_tokens'] = []
        sa['translation'] = gt[:3]
        sa['size'] = gt[3:6]
        # sa['rotation'] = [0, 0, 0, 1]
        sa['rotation'] = rpy2quaternion(0, 0, gt[6])
        sa['prev'] = ''
        sa['next'] = ''
        if len(info['num_lidar_pts']) > 0:
            num_lidar_pts = info['num_lidar_pts'][0]
        else:
            num_lidar_pts = 0
        sa['num_lidar_pts'] = int(num_lidar_pts)
        sa['num_radar_pts'] = 0
        sample_annotation.append(sa)

        ic = {}
        ic['token'] = 'instance_token_' + token + '_' + str(i)
        ic['category_token'] = 'vehicle.truck' 
        ic['nbr_annotations'] = 0
        ic['first_annotation_token'] = ''
        ic['last_annotation_token'] = ''
        instance.append(ic)
    
    return sample, sample_data, sample_annotation, instance     

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

sample = []
sample_data = []
sample_annotation = []
instance = []
for i in cdi_infos:
    sm, sd, sa, ic = info2json(i)
    sample.append(sm)
    sample_data += sd
    sample_annotation += sa
    instance += ic

sn = len(sample)
an = len(sample_annotation)
print(sn, an)
json.dump(sample, open(path1 + 'sample.json', 'w'))
json.dump(sample_data, open(path1 + 'sample_data.json', 'w'))
json.dump(sample_annotation, open(path1 + 'sample_annotation.json', 'w'), cls=NumpyEncoder)
json.dump(instance, open(path1 + 'instance.json', 'w'), cls=NumpyEncoder)


9657 3921


In [17]:
for i in cdi_infos[:1]:
    print(i['token'])

01a7d01c014f406f87b8fe11976c3d0a


In [8]:
scene = [{'token':'scene_token_0' ,
          'log_token':'',
          'nbr_samples':sn,
          'first_sample_token':'sample_token_samplescdi_cd1_16633753663_bin',
          'last_sample_token':'',
          'name':'scene-0001',
          'description':''}, ]
json.dump(scene, open(path1 + 'scene.json', 'w'))

sensor = [{'token': 'lidar_top', 'channel': 'LIDAR_TOP', 'modality': 'lidar'},
          {'token': 'cam_front', 'channel': 'CAM_FRONT', 'modality': 'camera'}]
json.dump(sensor, open(path1 + 'sensor.json', 'w'))

# json.dump(j_visibility, open(path1 + 'visibility.json', 'w'))
map = [{'category': 'semantic_prior', 'token': 'map', 'filename': '', 
        'log_tokens': []}]
json.dump(map, open(path1 + 'map.json', 'w'))
json.dump({}, open(path1 + 'log.json', 'w'))

ego_pose = [{'token': 'ego_pose_token', 
             'timestamp': 0, 
             'rotation': [0.0, 0.0, 0.0, 1],
             'translation': [0.0, 0.0, 0.0]}]
json.dump(ego_pose, open(path1 + 'ego_pose.json', 'w'))

category = [{'token': 'vehicle.truck', 'name': 'vehicle.truck', 'description': ''},
            {'token': 'movable_object.trafficcone', 'name': 'movable_object.trafficcone', 'description': ''}]
json.dump(category, open(path1 + 'category.json', 'w'))

calibrated_sensor = [{'token': 'calibrated_sensor_lidar_top_token', 
                      'sensor_token': 'lidar_top', 
                      'translation': [0.0, 0.0, 0.0], 
                      'rotation': [0.0, 0.0, 0.0, 1], 
                      'camera_intrinsic': []},
                     {'token': 'calibrated_sensor_cam_front_token', 
                      'sensor_token': 'cam_front', 
                      'translation': [0.0, 0.0, 0.0], 
                      'rotation': [0.0, 0.0, 0.0, 1], 
                      'camera_intrinsic': [[200, 0.0, 100], [0.0, 200, 100], [0.0, 0.0, 1.0]]}]
json.dump(calibrated_sensor, open(path1 + 'calibrated_sensor.json', 'w'))

json.dump({}, open(path1 + 'attribute.json', 'w'))

In [9]:
s = json.load(open(path1 + "sample_annotation.json", "rb"))
# s = json.load(open(path1 + "sample_data.json", "rb"))
# s = json.load(open(path1 + "sample.json", "rb"))
for i in s:
    print(i)

{'token': 'sample_annotation_token_samplescdi_cd1027_XT06_03_16660867453_bin_0', 'sample_token': 'sample_token_samplescdi_cd1027_XT06_03_16660867453_bin', 'instance_token': 'instance_token_samplescdi_cd1027_XT06_03_16660867453_bin_0', 'visibility_token': '4', 'attribute_tokens': [], 'translation': [-2.103334791113621, 9.169102819130687, -0.0840582875188863], 'size': [16.586099918141667, 3.3568702369960053, 3.820774793624878], 'rotation': [0.00472346044766677, 0.0, 0.0, 0.9999888443984759], 'prev': '', 'next': '', 'num_lidar_pts': 608, 'num_radar_pts': 0}
{'token': 'sample_annotation_token_samplescdi_cd1027_XT06_01_16660859604_bin_0', 'sample_token': 'sample_token_samplescdi_cd1027_XT06_01_16660859604_bin', 'instance_token': 'instance_token_samplescdi_cd1027_XT06_01_16660859604_bin_0', 'visibility_token': '4', 'attribute_tokens': [], 'translation': [64.40537111637171, 7.090868585579749, -0.6385703065342635], 'size': [8.99133267615445, 3.940564865709888, 4.924418253589765], 'rotation': [